In [1]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import sklearn as skl
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
from keras.models import Sequential
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
#### variabili globali
#### GIUSEPPE
#image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
image_path = '..\\..\\Untitled_Folder'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
csv_completo = 'dataframe_completo.csv'
csv_completo_2 = 'dataframe_completo_2.csv' 

In [3]:
def my_train_batch(dataset, batch, epochs, classifier):
    size = len(dataset)
    epoch=0
    cl1 = k.models.load_model(classifier)
    # train del classificatore un batch alla volta
    while size >= batch:
        # To get n random rows 
        samples = dataset.sample(n = batch)
        # splitting test and validation
        X_train, X_val, y_train, y_val = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
        
        train_img = np.asarray(load_images(X_train))
        val_img = np.asarray(load_images(X_val))
        
        #1hot encoder
        enc = OneHotEncoder(sparse=False)
        y_train_shaped = np.array(y_train).reshape(-1,1)
        y_val_shaped = np.array(y_val).reshape(-1,1)
        enc.fit(y_train_shaped)
        y_train_one = enc.transform(y_train_shaped)
        y_val_one = enc.transform(y_val_shaped)
        
        cl1.fit(train_img, y_train_one, epochs=epochs, shuffle=True, validation_data=(val_img, y_val_one), verbose=1)
        
        #pulizia memoria e reset per il prossimo ciclo
        del X_train, y_train_one, X_val, y_val_one, train_img, val_img
        dataset = dataset.drop(samples.index)
        size = len(dataset)
        del samples, enc
        epoch = epoch + 1
        print("fine epoca " + str(epoch) + ";\trestano " + str(size) + " campioni nel dataset")
        
    ######## ULTIMO CICLO CHE FINISCE IL DATASET
    samples = dataset.sample(n = size)
    # splitting test and validation
    X_train, X_val, y_train, y_val = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
    
    train_img = np.asarray(load_images(X_train))
    val_img = np.asarray(load_images(X_val))
    
    #1hot encoder
    enc = OneHotEncoder(sparse=False)
    y_train_shaped = np.array(y_train).reshape(-1,1)
    y_val_shaped = np.array(y_val).reshape(-1,1)
    enc.fit(y_train_shaped)
    y_train_one = enc.transform(y_train_shaped)
    y_val_one = enc.transform(y_val_shaped)
    
    cl1.fit(train_img, y_train_one, epochs=epochs, shuffle=True, validation_data=(val_img, y_val_one), verbose=1)
    
    del X_train, y_train_one, X_val, y_val_one, train_img, val_img
    dataset = dataset.drop(samples.index)
    size = len(dataset)
    del samples, enc
    epoch = epoch + 1
    print("fine ultima epoca")

    cl1.save(classifier)
    return

# dal dataframe ne estrae le immagini in un array, serve nel batch
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join(image_path, img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(np.asarray(img)) #########################################################################################
    return images

def new_classifier3():
    # Set the CNN model 
    input_shape = (150, 200, 3)
    num_classes = 7
    
    model = Sequential()
    model.add(Conv2D(20, kernel_size=(5, 5),activation='relu',padding = 'Same', input_shape = input_shape))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(140,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(50,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model

#def kaggle_classifier():
#    # Set the CNN model 
#    # my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
#    input_shape = (150, 200, 3)
#    num_classes = 7
#    
#    model = Sequential()
#    model.add(Conv2D(16, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
#    model.add(Conv2D(16,kernel_size=(3, 3), activation='relu',padding = 'Same',))
#    model.add(MaxPool2D(pool_size = (2, 2)))
#    model.add(Dropout(0.25))
#    
#    model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
#    model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
#    model.add(MaxPool2D(pool_size=(2, 2)))
#    model.add(Dropout(0.40))
#    
#    model.add(Flatten())
#    model.add(Dense(128, activation='relu'))
#    model.add(Dropout(0.5))
#    model.add(Dense(num_classes, activation='softmax'))
#    model.summary()
#    model.compile(optimizer='adadelta', loss='binary_crossentropy')
#    return model

In [4]:
# - dataset splittato 70 30
# - apro il 70
dataset = pd.read_csv(csv_completo_2, encoding = "ISO-8859-1")

mass = int(len(dataset)*0.7)
train = dataset.sample(n=mass)
train.to_csv('train07.csv', index=False)
test = dataset.drop(train.index)
test.to_csv('test03.csv', index=False)

del test

In [5]:
train = pd.read_csv("train07.csv", encoding = "ISO-8859-1")

In [5]:
# - creo cl_batch e cl_fit
n_cl = new_classifier3()
n_cl.save('cl_batch.h5')
n_cl.save('cl_fit.h5')
del n_cl

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 200, 20)      1520      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 100, 20)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 100, 20)       80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 100, 140)      25340     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 50, 140)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 37, 50, 140)       560       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 50, 50)       

In [6]:
# - carico in memoria le immagini
# - suddivido in train validation
# - 1hot encoding delle y
images = np.asarray(load_images(train["image_id"]))

X_train, X_val, y_train, y_val = train_test_split(images, train["dx"], test_size=0.30)

#1hot encoder
enc = OneHotEncoder(sparse=False)
y_train_shaped = np.array(y_train).reshape(-1,1)
y_val_shaped = np.array(y_val).reshape(-1,1)
enc.fit(y_train_shaped)
y_train_one = enc.transform(y_train_shaped)
y_val_one = enc.transform(y_val_shaped)

In [7]:
# FIT di cl_fit
cl_name = "cl_fit.h5"
cl_fit = k.models.load_model(cl_name)
cl_fit.fit(X_train, y_train_one, epochs=5, shuffle=True, validation_data=(X_val, y_val_one), verbose=1)
cl_fit.save(cl_name)

del X_train, X_val, y_train, y_val, y_train_shaped, y_val_shaped, y_train_one, y_val_one

Train on 23497 samples, validate on 10071 samples
Epoch 1/5
23497/23497 [==============================] - 2539s 108ms/step - loss: 0.3341 - val_loss: 1.4010
Epoch 2/5
23497/23497 [==============================] - 962s 41ms/step - loss: 0.2451 - val_loss: 0.3050
Epoch 3/5
23497/23497 [==============================] - 966s 41ms/step - loss: 0.2165 - val_loss: 0.4612
Epoch 4/5
23497/23497 [==============================] - 1046s 45ms/step - loss: 0.1952 - val_loss: 0.3335
Epoch 5/5
23497/23497 [==============================] - 1044s 44ms/step - loss: 0.1765 - val_loss: 0.6442


In [6]:
# - FIT di cl_batch su train
cl_name = "cl_batch.h5"
my_train_batch(dataset=train, batch=5000, epochs=5, classifier = cl_name)

Train on 3500 samples, validate on 1500 samples
Epoch 1/5
3500/3500 [==============================] - 131s 37ms/step - loss: 0.2116 - val_loss: 0.2419
Epoch 2/5
3500/3500 [==============================] - 145s 41ms/step - loss: 0.1515 - val_loss: 0.3860
Epoch 3/5
3500/3500 [==============================] - 144s 41ms/step - loss: 0.1235 - val_loss: 0.4172
Epoch 4/5
3500/3500 [==============================] - 148s 42ms/step - loss: 0.0953 - val_loss: 0.2461
Epoch 5/5
3500/3500 [==============================] - 147s 42ms/step - loss: 0.0772 - val_loss: 0.4013
fine epoca 1;	restano 28568 campioni nel dataset
Train on 3500 samples, validate on 1500 samples
Epoch 1/5
3500/3500 [==============================] - 145s 42ms/step - loss: 0.2162 - val_loss: 0.2962
Epoch 2/5
3500/3500 [==============================] - 151s 43ms/step - loss: 0.1551 - val_loss: 0.4884
Epoch 3/5
3500/3500 [==============================] - 149s 43ms/step - loss: 0.1179 - val_loss: 0.5801
Epoch 4/5
3500/3500 [==